In [ ]:
readme_text = """🧾 CAFA-6 Baseline: Protein Function Prediction using k-mers + TF-IDF + Logistic Regression

(Predicción de la función proteica mediante k-mers + TF-IDF + Regresión Logística)

⸻

👤 Author / Autor

Francisco José de la Corte López
Competition: CAFA-6 Protein Function Prediction
Version: Baseline v1 – October 2025

⸻

🧠 Objective / Objetivo

EN:
This notebook develops a baseline model for the CAFA-6 challenge. The goal is to predict Gene Ontology (GO) terms — Molecular Function (MF), Biological Process (BP), and Cellular Component (CC) — for each protein, based solely on its amino-acid sequence.

ES:
Este notebook desarrolla un modelo base (baseline) para la competición CAFA-6. El objetivo es predecir los términos de la Ontología Génica (GO) — Función Molecular (MF), Proceso Biológico (BP) y Componente Celular (CC) — de cada proteína a partir únicamente de su secuencia de aminoácidos.

⸻

⚙️ Methodology / Metodología

EN:
	1.	Data preparation: unified FASTA sequences and GO annotations into a clean train_merged.parquet.
	2.	Feature extraction: split sequences into k-mers (k=3) and applied TF-IDF vectorization.
	3.	Modeling: trained One-Vs-Rest Logistic Regression separately for MF, BP, and CC.
	4.	Validation: 80/20 protein split, evaluated with F1 (micro/macro).
	5.	Submission: top-200 GO terms per ontology combined into submission_baseline.tsv.

ES:
	1.	Preparación de datos: unificación de secuencias FASTA y anotaciones GO en un único archivo limpio (train_merged.parquet).
	2.	Extracción de características: división de las secuencias en k-mers (k=3) y aplicación de vectorización TF-IDF.
	3.	Modelado: entrenamiento de un modelo de Regresión Logística One-Vs-Rest para cada subontología (MF, BP y CC).
	4.	Validación: división 80/20 por proteína y evaluación con F1 (micro y macro).
	5.	Generación de resultados: selección de los 200 términos GO con mayor probabilidad por ontología y combinación en submission_baseline.tsv.

⸻

🤖 Use of AI Assistance / Uso de Inteligencia Artificial

EN:
Parts of the code and documentation were created with help from ChatGPT (GPT-5) as a coding assistant.
All data processing, execution, validation, and interpretation were carried out by the author.
This use of AI complies with Kaggle’s and CAFA’s ethical rules.

ES:
Parte del código y de la documentación se elaboró con el apoyo de un asistente de IA (ChatGPT – GPT-5).
Todas las tareas de diseño, ejecución, validación e interpretación fueron realizadas por el autor.
El uso de IA cumple con las normas éticas de Kaggle y de la organización CAFA.

⸻

🔬 Next Steps / Próximos Pasos

EN:
	•	Threshold tuning per ontology (F-max).
	•	Hierarchical GO propagation (respecting DAG structure).
	•	Use of pretrained embeddings (ESM-2 / ProtT5).
	•	Ensemble combination of MF + BP + CC models for final submission.

ES:
	•	Ajuste de umbrales por ontología (F-max).
	•	Propagación jerárquica de términos GO (respeto de la estructura DAG).
	•	Uso de embeddings proteicos pre-entrenados (ESM-2 / ProtT5).
	•	Ensemble final combinando los modelos MF + BP + CC.

⸻

📜 Notes / Notas

EN:
No post-2025 experimental data or future annotations were used. All computations rely solely on publicly available data at the competition start.

ES:
No se ha utilizado información experimental publicada después del 15 de octubre de 2025.
Todos los cálculos se basan exclusivamente en datos públicos disponibles en la fecha de inicio de la competición.

⸻

with open("/kaggle/working/outputs/baseline_readme.txt", "w") as f:
    f.write(readme_text)
print("✅ README guardado en /kaggle/working/outputs/baseline_readme.txt")# ==========================================================


# CONFIGURACIÓN INICIAL — CAFA 6 Protein Function Prediction
# ==========================================================
# Esta celda prepara las rutas y copia los datos de la competición
# a una carpeta de trabajo accesible en /kaggle/working/data

import os, shutil
from pathlib import Path

# Ruta base de la competición montada automáticamente
SRC = "/kaggle/input/cafa-6-protein-function-prediction"
DEST = "/kaggle/working/data"
os.makedirs(DEST, exist_ok=True)

# Copiamos y renombramos los archivos clave
mapping = {
    f"{SRC}/Train/train_sequences.fasta": f"{DEST}/train.fasta",
    f"{SRC}/Train/train_terms.tsv": f"{DEST}/train_annotations.tsv",
    f"{SRC}/Test/testsuperset.fasta": f"{DEST}/test_superset.fasta",
    f"{SRC}/IA.tsv": f"{DEST}/IA.tsv",
}
for src, dst in mapping.items():
    if os.path.exists(src):
        shutil.copy(src, dst)

# Mostramos lo que hay en la carpeta de trabajo
print("✅ Datos preparados en:", DEST)
!ls -lh $DEST

In [ ]:
# ===============================================
# EXPLORACIÓN INICIAL DE train_annotations.tsv
# ===============================================
import pandas as pd
import matplotlib.pyplot as plt

DATA_DIR = "/kaggle/working/data"

# Cargar las anotaciones
ann = pd.read_csv(f"{DATA_DIR}/train_annotations.tsv", sep='\t', header=None,
                  names=['protein_id', 'go_id', 'ontology'])

print("✅ Anotaciones cargadas:", ann.shape)
print(ann.head())

# Resumen general
n_proteins = ann['protein_id'].nunique()
n_terms = ann['go_id'].nunique()
print(f"\nNúmero de proteínas únicas: {n_proteins:,}")
print(f"Número de términos GO únicos: {n_terms:,}")

# Conteo por subontología
ont_count = ann['ontology'].value_counts()
print("\nAnotaciones por subontología:")
print(ont_count)

# Visualización rápida
ont_count.plot(kind='bar', color=['#5DADE2', '#58D68D', '#F5B041'])
plt.title("Distribución de anotaciones por subontología (MF / BP / CC)")
plt.ylabel("Número de anotaciones")
plt.show()

# Conteo de términos por frecuencia (cola larga)
term_freq = ann['go_id'].value_counts()
print("\nEjemplo de términos más frecuentes:")
print(term_freq.head(10))

Métrica
Valor
✅ Anotaciones cargadas
537.028 filas (protein–GO–ontology)
🧬 Proteínas únicas
82.405
🧠 Términos GO únicos
26.126
📊 Subontologías
P (Biological Process) → 250.805  C (Cellular Component) → 157.770  F (Molecular Function) → 128.452
Y la gráfica de barras muestra justo eso:
BP domina (P), seguida por CC (C) y MF (F).
Además, los términos GO más frecuentes (GO:0005515, GO:005634, etc.) coinciden con lo esperado:
	•	GO:0005515 = protein binding → uno de los términos más comunes de la ontología MF.
Así que el análisis es coherente 💯.

In [ ]:
# ===============================================
# EXPLORACIÓN DE SECUENCIAS (train.fasta)
# ===============================================
import gzip
import numpy as np

def read_fasta(fp):
    """Generador simple de FASTA (soporta .gz)"""
    opener = gzip.open if str(fp).endswith('.gz') else open
    header, seq = None, []
    with opener(fp, 'rt') as f:
        for line in f:
            line = line.strip()
            if not line:
                continue
            if line.startswith('>'):
                if header:
                    yield header, ''.join(seq)
                header = line[1:].split()[0]
                seq = []
            else:
                seq.append(line)
        if header:
            yield header, ''.join(seq)

lengths = []
for _, seq in read_fasta(f"{DATA_DIR}/train.fasta"):
    lengths.append(len(seq))

print(f"✅ Secuencias leídas: {len(lengths):,}")
print(f"Longitud mínima: {np.min(lengths)}")
print(f"Longitud máxima: {np.max(lengths)}")
print(f"Longitud media: {np.mean(lengths):.1f}")
print(f"Mediana: {np.median(lengths):.1f}")

plt.hist(lengths, bins=50, color="#5DADE2", edgecolor="black")
plt.title("Distribución de longitudes de secuencia (train.fasta)")
plt.xlabel("Longitud (aminoácidos)")
plt.ylabel("Número de proteínas")
plt.show()

In [ ]:
# ==========================================================
# UNIÓN DE SECUENCIAS Y ANOTACIONES
# ==========================================================
from tqdm import tqdm

# Convertimos el FASTA a DataFrame
records = []
for pid, seq in tqdm(read_fasta(f"{DATA_DIR}/train.fasta")):
    records.append((pid, seq))
df_seq = pd.DataFrame(records, columns=["protein_id", "sequence"])
print("Proteínas con secuencia:", len(df_seq))

# Comprobamos que todas las proteínas de las anotaciones están en el FASTA
missing = set(ann['protein_id']) - set(df_seq['protein_id'])
print(f"Proteínas anotadas sin secuencia: {len(missing)}")

# Unimos las anotaciones (uno-a-muchos)
df_merged = ann.merge(df_seq, on="protein_id", how="left")

print("\nDimensiones del dataframe final:", df_merged.shape)
print(df_merged.head())

# Guardamos para futuros pasos
df_merged.to_parquet(f"{DATA_DIR}/train_merged.parquet", index=False)
print("✅ Guardado train_merged.parquet")

In [ ]:
# Releer anotaciones indicando que SÍ tienen cabecera
import pandas as pd

DATA_DIR = "/kaggle/working/data"

ann = pd.read_csv(f"{DATA_DIR}/train_annotations.tsv",
                  sep="\t", header=0,  # <- hay cabecera
                  names=["protein_id","go_id","ontology"])  # nombres unificados

# Limpieza básica
ann = ann.dropna(subset=["protein_id","go_id","ontology"])
ann = ann.astype({"protein_id": str, "go_id": str, "ontology": str})

print("✅ Anotaciones recargadas:", ann.shape)
print(ann.head())

print("\nRecuento por ontología (ya sin 'aspect'):")
print(ann["ontology"].value_counts())

In [ ]:
import gzip, re
from tqdm import tqdm

def read_fasta_with_header(fp):
    """Devuelve (header_completo_sin_'>', secuencia)"""
    opener = gzip.open if str(fp).endswith(".gz") else open
    header, seq = None, []
    with opener(fp, "rt") as f:
        for line in f:
            line = line.strip()
            if not line:
                continue
            if line.startswith(">"):
                if header:
                    yield header, "".join(seq)
                header = line[1:]  # sin '>'
                seq = []
            else:
                seq.append(line)
        if header:
            yield header, "".join(seq)

def extract_uniprot_accession(header: str) -> str:
    """
    Intenta extraer el accesión UniProt del header.
    Casos típicos: 'sp|Q5W0B1|...', 'tr|A0A123|...', o encabezados simples.
    """
    first_token = header.split()[0]
    # Si hay tuberías, busca un token que parezca accesión
    if "|" in first_token:
        for part in first_token.split("|"):
            if re.fullmatch(r"[A-NR-Z0-9]{6,10}", part):  # patrón accesión
                return part
        # si no encontró, devuelve el último trozo
        return first_token.split("|")[-1]
    # Sin tuberías: intenta extraer un patrón de accesión
    m = re.search(r"[A-NR-Z0-9]{6,10}", first_token)
    return m.group(0) if m else first_token

# Convertimos el FASTA a DataFrame usando el accesión
records = []
for header, seq in tqdm(read_fasta_with_header(f"{DATA_DIR}/train.fasta")):
    acc = extract_uniprot_accession(header)
    records.append((acc, seq))

df_seq = pd.DataFrame(records, columns=["protein_id","sequence"])
print("✅ Proteínas con secuencia (IDs parseados):", len(df_seq))
print(df_seq.head())

# Comprobación de cobertura
missing = set(ann["protein_id"]) - set(df_seq["protein_id"])
print("Proteínas anotadas sin secuencia (debería ser ~0):", len(missing))

# Unión correcta
df_merged = ann.merge(df_seq, on="protein_id", how="left")
print("\nDimensiones del dataframe final:", df_merged.shape)
print(df_merged.head())

# Guardar limpio
df_merged.to_parquet(f"{DATA_DIR}/train_merged.parquet", index=False)
print("✅ Guardado train_merged.parquet (IDs alineados)")

In [ ]:
import re, gzip
from tqdm import tqdm
import pandas as pd

DATA_DIR = "/kaggle/working/data"

# Patrones de accesión UniProt: 6 caracteres (clásico) o 10 (A0A…)
ACC_REGEX = re.compile(r"(?:[OPQ][0-9][A-Z0-9]{3}[0-9]|[A-NR-Z][0-9][A-Z0-9]{3}[0-9]|[A-Z0-9]{10})")

def read_fasta_headers(fp):
    opener = gzip.open if str(fp).endswith(".gz") else open
    header, seq = None, []
    with opener(fp, "rt") as f:
        for line in f:
            line = line.strip()
            if not line:
                continue
            if line.startswith(">"):
                if header:
                    yield header, "".join(seq)
                header = line[1:]
                seq = []
            else:
                seq.append(line)
        if header:
            yield header, "".join(seq)

def extract_accession(header: str) -> str:
    """
    Reglas:
    1) Si hay tuberías (sp|ACC|ENTRY / tr|ACC|ENTRY), toma el campo que cumpla ACC_REGEX.
    2) Si no, busca el primer match ACC_REGEX en TODO el header.
    3) Si falla, devuelve el primer token (caso residual).
    """
    # Caso con tuberías
    if "|" in header:
        for part in header.split("|"):
            m = ACC_REGEX.fullmatch(part)
            if m:
                return m.group(0)
    # Búsqueda en todo el header
    m = ACC_REGEX.search(header)
    if m:
        return m.group(0)
    # Fallback: primer token
    return header.split()[0]

In [ ]:
# FASTA -> DataFrame con accesión correcto
records = []
for header, seq in tqdm(read_fasta_headers(f"{DATA_DIR}/train.fasta")):
    acc = extract_accession(header)
    records.append((acc, seq))
df_seq = pd.DataFrame(records, columns=["protein_id","sequence"])
print("✅ Proteínas con secuencia (IDs parseados):", len(df_seq))
print(df_seq.head())

# Recargar anotaciones con cabecera real (por si no lo hiciste antes)
ann = pd.read_csv(f"{DATA_DIR}/train_annotations.tsv",
                  sep="\t", header=0,
                  names=["protein_id","go_id","ontology"])
ann = ann.dropna(subset=["protein_id","go_id","ontology"]).astype(str)

# Comprobación de cobertura
missing = set(ann["protein_id"]) - set(df_seq["protein_id"])
print("Proteínas anotadas sin secuencia (objetivo ~0):", len(missing))

# Unir
df_merged = ann.merge(df_seq, on="protein_id", how="left")
print("\nDimensiones del dataframe final:", df_merged.shape)
print(df_merged.head())

# Guardar limpio
df_merged.to_parquet(f"{DATA_DIR}/train_merged.parquet", index=False)
print("✅ Guardado train_merged.parquet (IDs alineados de verdad)")

HASTA AHORA TENEMOS:Fase
Estado
Archivo resultante
Descarga y organización de datos
✅ completada
/kaggle/working/data
EDA de anotaciones (MF/BP/CC)
✅ completada
train_annotations.tsv
EDA de secuencias (longitudes, distribución)
✅ completada
train.fasta
Unión limpia (proteína + GO + ontología + secuencia)
✅ completada
train_merged.parquet


In [ ]:
# ===============================================
# HELPERS: k-mers, corpus y utilidades
# ===============================================
from pathlib import Path
import numpy as np
import pandas as pd
from collections import defaultdict, Counter
from tqdm import tqdm

import matplotlib.pyplot as plt

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, precision_score, recall_score

DATA_DIR = Path("/kaggle/working/data")
OUT_DIR  = Path("/kaggle/working/outputs"); OUT_DIR.mkdir(exist_ok=True, parents=True)

def seq_to_kmers(seq: str, k=3):
    return [seq[i:i+k] for i in range(0, max(0, len(seq)-k+1))]

def make_docs(df_seq: pd.DataFrame, k=3):
    # df_seq: columnas ['protein_id','sequence']
    docs = []
    ids  = []
    for pid, seq in zip(df_seq['protein_id'].values, df_seq['sequence'].values):
        ids.append(pid)
        docs.append(" ".join(seq_to_kmers(seq, k=k)))
    return ids, docs

def df_labels_from_ann(df_ann: pd.DataFrame, ontology: str):
    sub = df_ann[df_ann['ontology'] == ontology].copy()
    labels = sub.groupby('protein_id')['go_id'].apply(set).reset_index()
    return labels  # columnas: protein_id | set(go_ids)

def split_by_protein(df_seq: pd.DataFrame, random_state=42, test_size=0.2):
    # split simple a nivel de proteína
    train_ids, valid_ids = train_test_split(
        df_seq['protein_id'].unique(), test_size=test_size, random_state=random_state, shuffle=True
    )
    return set(train_ids), set(valid_ids)

def show_scores(y_true, y_prob, thr=0.5, name=""):
    y_pred = (y_prob >= thr).astype(int)
    micro = f1_score(y_true, y_pred, average="micro", zero_division=0)
    macro = f1_score(y_true, y_pred, average="macro", zero_division=0)
    prec  = precision_score(y_true, y_pred, average="micro", zero_division=0)
    rec   = recall_score(y_true, y_pred, average="micro", zero_division=0)
    print(f"[{name}] F1 micro={micro:.4f} | F1 macro={macro:.4f} | P={prec:.4f} | R={rec:.4f}")
    return {"f1_micro": micro, "f1_macro": macro, "prec": prec, "rec": rec}

In [ ]:
# ===============================================
# CARGA DE DATOS UNIFICADOS (hecho en pasos previos)
# ===============================================
df = pd.read_parquet(DATA_DIR / "train_merged.parquet")
print(df.shape, df.head())

# Creamos dataframe de secuencias únicas
df_seq = df[['protein_id','sequence']].drop_duplicates().reset_index(drop=True)
print("Proteínas (únicas) con secuencia:", df_seq.shape[0])

# Probaremos k=3 (rápido y robusto). Puedes cambiar a k=4 más adelante.
KMER = 3

# Split por proteína
train_ids, valid_ids = split_by_protein(df_seq, test_size=0.2, random_state=42)
print("Train proteins:", len(train_ids), " | Valid proteins:", len(valid_ids))

In [ ]:
# ===============================================
# ENTRENAMIENTO + VALIDACIÓN POR SUBONTOLOGÍA
# ===============================================
from collections import namedtuple
ModelPack = namedtuple("ModelPack", ["vectorizer","clf","mlb","terms"])

models = {}
metrics = {}

for ont, ont_name in [("F","MF"), ("P","BP"), ("C","CC")]:
    print("\n","="*20, f"{ont_name}", "="*20)
    # Etiquetas por proteína en esta ontología
    lab = df_labels_from_ann(df, ontology=ont)
    # Merge con secuencias
    sub = df_seq.merge(lab, on="protein_id", how="left")
    sub['go_id'] = sub['go_id'].apply(lambda x: x if isinstance(x, set) else set())
    
    # Split
    train_df = sub[sub['protein_id'].isin(train_ids)].reset_index(drop=True)
    valid_df = sub[sub['protein_id'].isin(valid_ids)].reset_index(drop=True)
    print("train:", train_df.shape, "valid:", valid_df.shape)
    
    # Documentos k-mer
    tr_ids, tr_docs = make_docs(train_df[['protein_id','sequence']], k=KMER)
    va_ids, va_docs = make_docs(valid_df[['protein_id','sequence']], k=KMER)
    
    # Vectorizador TF-IDF (control de tamaño)
    tfidf = TfidfVectorizer(min_df=2, max_df=0.9, max_features=200_000)
    X_tr = tfidf.fit_transform(tr_docs)
    X_va = tfidf.transform(va_docs)
    
    # Binarizador multilabel
    mlb = MultiLabelBinarizer(sparse_output=False)
    Y_tr = mlb.fit_transform(train_df['go_id'].tolist())
    Y_va = mlb.transform(valid_df['go_id'].tolist())
    print("n_labels:", Y_tr.shape[1])
    
    # Modelo (One-vs-Rest LR con solver liblinear/saga según tamaño)
    clf = OneVsRestClassifier(
        LogisticRegression(max_iter=200, n_jobs=1, solver="liblinear")
    )
    clf.fit(X_tr, Y_tr)
    
    # Validación simple (umbral 0.5; luego haremos tuning)
    Yp_va = clf.predict_proba(X_va)
    metrics[ont] = show_scores(Y_va, Yp_va, thr=0.5, name=f"{ont_name}")
    
    models[ont] = ModelPack(vectorizer=tfidf, clf=clf, mlb=mlb, terms=list(mlb.classes_))

metrics

In [ ]:
# ===============================================
# PREDICCIÓN EN TEST + ARCHIVO DE ENVÍO
# ===============================================
import gzip, re

TEST_FASTA = DATA_DIR / "test_superset.fasta"
SUBMIT_FP  = OUT_DIR / "submission_baseline.tsv"

# Reutilizamos el lector de FASTA con extracción de accesión (del paso previo)
ACC_REGEX = re.compile(r"(?:[OPQ][0-9][A-Z0-9]{3}[0-9]|[A-NR-Z][0-9][A-Z0-9]{3}[0-9]|[A-Z0-9]{10})")

def read_fasta_headers(fp):
    opener = gzip.open if str(fp).endswith(".gz") else open
    header, seq = None, []
    with opener(fp, "rt") as f:
        for line in f:
            line = line.strip()
            if not line: 
                continue
            if line.startswith(">"):
                if header:
                    yield header, "".join(seq)
                header = line[1:]
                seq = []
            else:
                seq.append(line)
        if header:
            yield header, "".join(seq)

def extract_accession(header: str) -> str:
    if "|" in header:
        for part in header.split("|"):
            if ACC_REGEX.fullmatch(part):
                return part
    m = ACC_REGEX.search(header)
    if m:
        return m.group(0)
    return header.split()[0]

# Construimos documentos test (una pasada)
test_records = []
for h, s in tqdm(read_fasta_headers(TEST_FASTA)):
    acc = extract_accession(h)
    test_records.append((acc, s))
df_test = pd.DataFrame(test_records, columns=["protein_id","sequence"])
print("Test proteins:", df_test.shape)

# Predicción por ontología
per_protein_scores = defaultdict(list)  # protein_id -> list[(go_id, score)]

TOP_K_PER_ONTO = 200  # puedes ajustar (el límite global de Kaggle es 1500 por proteína sumando MF/BP/CC)

for ont, pack in models.items():
    print(f"Predicting {ont} ...")
    ids, docs = make_docs(df_test, k=KMER)
    X = pack.vectorizer.transform(docs)
    Yp = pack.clf.predict_proba(X)  # shape: [n_prot, n_terms_ont]
    
    # Para cada proteína, quedarnos con top-k de esta ontología
    for i, pid in enumerate(ids):
        row = Yp[i]
        if row.ndim == 0:  # corner case
            continue
        top_idx = np.argsort(-row)[:TOP_K_PER_ONTO]
        for j in top_idx:
            score = float(row[j])
            if score <= 0.0: 
                continue
            go_id = pack.terms[j]
            per_protein_scores[pid].append((go_id, score))

# Escribir submission (tab-separated, score con hasta 3 cifras significativas, sin header)
n_lines = 0
with open(SUBMIT_FP, "w", encoding="utf-8") as f:
    for pid, items in per_protein_scores.items():
        # ordenar globalmente (MF+BP+CC juntos) y podar duro si quisieras (ej. 600)
        items.sort(key=lambda x: x[1], reverse=True)
        for go_id, score in items:
            s = f"{score:.3g}"
            if float(s) <= 0.0:
                s = "1e-6"
            f.write(f"{pid}\t{go_id}\t{s}\n")
            n_lines += 1

print(f"✅ Submission escrito: {SUBMIT_FP} ({n_lines:,} líneas)")
!head -n 10 {SUBMIT_FP}

In [ ]:
# Guardar métricas y parámetros del baseline
pd.DataFrame(metrics).to_csv(OUT_DIR / "baseline_metrics.csv")
with open(OUT_DIR / "baseline_readme.txt","w") as w:
    w.write(
        f"Baseline k-mer={KMER}, TF-IDF(max_features=200k), OvR-LogReg(liblinear)\n"
        f"TOP_K_PER_ONTO={TOP_K_PER_ONTO}\n"
        f"Valid split: 80/20 by protein\n"
    )
print("Artefactos guardados en:", OUT_DIR)

In [ ]:
# ===============================================
# TUNING DE UMBRAL (F-max) POR SUBONTOLOGÍA
# ===============================================
import numpy as np
from sklearn.metrics import f1_score

best_thresholds = {}

for ont, pack in models.items():
    print(f"\nTuning threshold for {ont} ...")
    
    # Obtener validación
    lab = df_labels_from_ann(df, ontology=ont)
    sub = df_seq.merge(lab, on="protein_id", how="left")
    sub['go_id'] = sub['go_id'].apply(lambda x: x if isinstance(x, set) else set())
    valid_df = sub[sub['protein_id'].isin(valid_ids)].reset_index(drop=True)
    ids, docs = make_docs(valid_df[['protein_id','sequence']], k=KMER)
    
    X_val = pack.vectorizer.transform(docs)
    Y_true = pack.mlb.transform(valid_df['go_id'].tolist())
    Y_prob = pack.clf.predict_proba(X_val)
    
    best_f1 = 0
    best_thr = 0.5
    for thr in np.linspace(0.1, 0.9, 17):  # prueba cada 0.05
        Y_pred = (Y_prob >= thr).astype(int)
        f1 = f1_score(Y_true, Y_pred, average="micro", zero_division=0)
        if f1 > best_f1:
            best_f1, best_thr = f1, thr
    best_thresholds[ont] = {"thr": best_thr, "f1_val": best_f1}
    print(f"Best threshold for {ont}: {best_thr:.2f} → F1={best_f1:.4f}")

best_thresholds

In [ ]:
# ===============================================
# PROPAGACIÓN JERÁRQUICA SIMPLIFICADA
# ===============================================
# NOTA: esta es una versión aproximada sin DAG externo (útil como baseline jerárquico)
# Más adelante podemos usar obonet para cargar el GO completo.

from collections import defaultdict

# Simulación de un "mapa padre" basado en IA.tsv (prepararemos el real en la siguiente fase)
parent_map = defaultdict(list)
# parent_map['GO:0005524'] = ['GO:0000166']  # ejemplo manual

def propagate_predictions(pred_dict, parent_map):
    """propaga puntuaciones desde los hijos a los padres"""
    for pid, items in pred_dict.items():
        scores = dict(items)
        for go_id, score in list(scores.items()):
            for parent in parent_map.get(go_id, []):
                scores[parent] = max(scores.get(parent, 0), score)
        pred_dict[pid] = list(scores.items())
    return pred_dict